## This tutorial covers how to organize your own images for use with CellOrganizer.  Typically, telling CellOrganizer what files to use to construct a model is done using wildcards (e.g., “Expt1Sample*Nuclear.tif”), and it is helpful to organize your files to facilitate this.  

### Run the cell below to set up basic parameters:

In [1]:
from cellorganizer.tools import img2slml
from cellorganizer.tools import slml2info
import datetime
import glob
import urllib.request
import os
import uuid
from IPython.core.display import display, HTML, Image

model_name = "Module1b"
module_name = model_name
topdir = "/home/murphylab/cellorganizer/local/results"
outputdir = topdir + "/" + module_name
if not os.path.exists(topdir):
    os.makedirs(topdir)
os.chdir(topdir)
os.system("ls")
if not os.path.exists(outputdir):
    os.makedirs(outputdir)
os.chdir(outputdir)
os.system("ls")
if not os.path.exists(model_name):
    os.makedirs(model_name)
os.chdir(model_name)
os.system("ls")

options = {}

#set general options
options["model.name"] = module_name
options["model.filename"] = module_name + '.mat'
options["output_filename"] = module_name 
options["verbose"] = 0
options["debug"] = 0
options["display"] = 0
options["model.id"] = str (uuid.uuid1())
options["downsampling"] = [5, 5, 1]
options["model.resolution"] = [0.049, 0.049, 0.2000]
#options["if_skip_cell_nuclear_model"] = 1


# postprocess of parameterization: alignment
# options["model.spharm_rpdm.postprocess"] = 'true'
# # alignment method: 'major_axis' or 'foe'
# options["model.spharm_rpdm.alignment_method"] = 'major_axis'
# # plane of rotation: 'xy', 'xz', 'yz' or 'xyz'
# options["model.spharm_rpdm.rotation_plane"] = 'xy'
# # degree of the descriptor
# options["model.spharm_rpdm.maxDeg"] = 31
# # latent dimension for the model
# options["model.spharm_rpdm.latent_dim"] = 15
# options["model.spharm_rpdm.segminnucfraction"] = 0.1

### Set up what kind of model is being created:

In [2]:
#set options that control what kind of model is created
options["train.flag"] = 'framework'
options["cell.class"] = 'cell_membrane'
#simplest cell shape model - ratio to nuclear model
#options["cell.type"] = 'ratio'
#best cell shape model - spharm_rpdm - see demo3D50 and demo3D52 for more options
options["cell.type"] = 'spharm_rpdm'
# if we are going to include nuclear/DNA images
options["nucleus.class"] = 'nuclear_membrane'
#simplest nuclear shape model - cylindrical surface
#options["nucleus.type"] = 'cylindrical_surface'
#best nuclear shape model - spharm_rpdm - see demo3D52 for more options
options["nucleus.type"] = 'spharm_rpdm'
#options["model.spharm_rpdm.components"] = {'cell'}
options["model.spharm_rpdm.components"] = {'nuc', 'cell'}
options["hd_thresh"] = 1.0

### Put your files into a directory, ideally seperated by channel (cell, nuclear, protein). So that one directory holds identical files of the same channel.

### Single file for each cell and channel:

In [3]:
options["masks"] = []

directory = '/home/murphylab/cellorganizer/local/images/HeLa/3D/processed/'
dnaImagesDirectoryPath = []
cellImagesDirectoryPath = []

file_pattern = 'LAM_cell1?_ch1_t1.tif'
for name in glob.glob(directory + file_pattern):
    cellImagesDirectoryPath.append(name)
    
file_pattern = 'LAM_cell1?_ch0_t1.tif'
for name in glob.glob(directory + file_pattern):
    dnaImagesDirectoryPath.append(name)
    
cellImagesDirectoryPath.sort()
dnaImagesDirectoryPath.sort()

cellImagesDirectoryPath = cellImagesDirectoryPath[0:3]
dnaImagesDirectoryPath = dnaImagesDirectoryPath[0:3]

### Multiple Cells and single channel with binary mask

In [4]:
# options["masks"] = []

# directory = '/home/murphylab/cellorganizer/local/images/HeLa/3D/processed/'
# dnaImagesDirectoryPath = []
# cellImagesDirectoryPath = []

# file_pattern = 'LAM_cell1?_mask_t1.tif'
# for name in glob.glob(directory + file_pattern):
#     options["masks"].append(name)

# file_pattern = 'LAM_cell1?_ch1_t1.tif'
# for name in glob.glob(directory + file_pattern):
#     cellImagesDirectoryPath.append(name)
    
# file_pattern = 'LAM_cell1?_ch0_t1.tif'
# for name in glob.glob(directory + file_pattern):
#     dnaImagesDirectoryPath.append(name)

# cellImagesDirectoryPath.sort()
# dnaImagesDirectoryPath.sort()
# options["masks"].sort()

# cellImagesDirectoryPath = cellImagesDirectoryPath[0:3]
# dnaImagesDirectoryPath = dnaImagesDirectoryPath[0:3]

### Multiple cells with multiple channels using OME-TIFF files with ROIs (Regions of Interests)

In [5]:
#options["masks"] = []

#directory = '/home/murphylab/cellorganizer/local/images/ometiff_with_rois/'
#dnaImagesDirectoryPath = []
#cellImagesDirectoryPath = []

#file_pattern = 'hela_3d_processed_lam_cell?.ome.tif'
#for name in glob.glob(directory + file_pattern):
#    cellImagesDirectoryPath.append(name + ':1:0')
#    dnaImagesDirectoryPath.append(name + ':2:0')

#cellImagesDirectoryPath.sort()
#dnaImagesDirectoryPath.sort()

In [6]:
answer = img2slml('3D', dnaImagesDirectoryPath, cellImagesDirectoryPath, [], options)

### Feel free to edit the three different options with the path to your own images!

In [7]:
optionsinfo={}
answer = slml2info([module_name + ".mat"],optionsinfo)

HTML(filename="index.html")

PermissionError: [Errno 1] Operation not permitted: 'input.txt'